In [19]:
import pandas as pd

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings("ignore")

import hopsworks

project = hopsworks.login() 

fs = project.get_feature_store() 

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5316
Connected. Call `.close()` to terminate connection gracefully.


In [20]:
feature_view = fs.get_feature_view(
    name = 'oslo_air_quality_fv',
    version = 1
)

train_data = feature_view.get_training_data(1)[0]

train_data.head()

,aqi,date,tempmax,tempmin,temp,dew,humidity,precip,precipcover,snow,snowdepth,windgust,windspeed,winddir,cloudcover,visibility,uvindex,conditions
0,32.0,1609776000000,-1.753477,-2.116327,-1.994985,-1.701971,0.697775,-0.101186,-0.569975,-0.257916,0.545417,-0.652988,-0.463010,-1.214297,0.263621,0.972892,-0.644401,4
1,30.0,1616515200000,-0.045975,-0.504283,-0.444708,-0.518692,-0.135001,-0.332334,-0.089113,-0.152673,4.524819,0.149146,0.718125,0.490510,0.040931,-0.285896,0.031588,15
2,30.0,1617465600000,-0.288174,-0.618410,-0.498631,-0.661256,-0.443436,0.903054,0.551780,-0.257916,1.446829,1.486037,1.958318,0.608848,-1.055713,0.136125,0.031588,15
3,31.0,1626710400000,1.443548,1.179090,1.456066,0.636074,-1.590815,-0.334401,-0.409944,-0.257916,-0.553865,-0.507840,0.112794,1.165406,-0.244785,1.176627,1.383567,2
4,26.0,1657382400000,1.068140,0.979368,1.078608,0.265408,-1.603153,-0.334711,-0.730391,-0.257916,-0.553865,0.882526,0.836239,1.608249,-0.862435,-1.180873,1.721562,7


In [21]:
train_data = train_data.sort_values(by=["date"], ascending=[False]).reset_index(drop=True)
train_data["aqi_next_day"] = train_data['aqi'].shift(1)

train_data.head(5)

X = train_data.drop(columns=["date"]).fillna(0)
y = X.pop("aqi_next_day")

gb = GradientBoostingRegressor()
gb.fit(X, y)

GradientBoostingRegressor()

In [22]:
f1_score(y.astype('int'),[int(pred) for pred in gb.predict(X)],average='micro')

y.iloc[4:10].values

pred_df = pd.DataFrame({
    'aqi_real': y.iloc[4:10].values,
    'aqi_pred': map(int, gb.predict(X.iloc[4:10]))
}
)
pred_df

,aqi_real,aqi_pred
0,23.0,32
1,39.0,34
2,24.0,30
3,28.0,29
4,15.0,26
5,55.0,45


In [23]:
mr = project.get_model_registry()

from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X)
output_schema = Schema(y)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

model_schema.to_dict()

import joblib

joblib.dump(gb, 'model.pkl')

model = mr.sklearn.create_model(
    name="air_quality_model",
    metrics={"f1": "0.5"},
    description="Gradient Boost Regressor.",
    input_example=X.sample(),
    model_schema=model_schema
)

model.save('model.pkl')

Connected. Call `.close()` to terminate connection gracefully.


  0%|          | 0/6 [00:00<?, ?it/s]

Model created, explore it at https://c.app.hopsworks.ai:443/p/5316/models/air_quality_model/3


Model(name: 'air_quality_model', version: 3)